In [1]:
from pprint import pprint

import tensorflow as tf
tf.__version__
tf.config.run_functions_eagerly(True)

In [2]:
BATCH_SIZE = 4

In [3]:
samples = tf.data.experimental.make_csv_dataset("data/random-games/samples.csv", batch_size=BATCH_SIZE)
# actions = tf.data.experimental.make_csv_dataset("data/random-games/actions.csv", batch_size=BATCH_SIZE)
rewards = tf.data.experimental.make_csv_dataset("data/random-games/rewards.csv", batch_size=BATCH_SIZE)

/home/bcollazo/.local/share/virtualenvs/catanatron-fU0XdcvU/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


In [4]:
dataset = tf.data.Dataset.zip((samples, rewards))
for element in dataset:
    pprint(element)
    break

(OrderedDict([('BANK_BRICK',
               <tf.Tensor: shape=(4,), dtype=float32, numpy=array([18., 15., 17., 16.], dtype=float32)>),
              ('BANK_DEV_CARDS',
               <tf.Tensor: shape=(4,), dtype=float32, numpy=array([25.,  5., 23.,  0.], dtype=float32)>),
              ('BANK_ORE',
               <tf.Tensor: shape=(4,), dtype=float32, numpy=array([17., 15., 15., 15.], dtype=float32)>),
              ('BANK_SHEEP',
               <tf.Tensor: shape=(4,), dtype=float32, numpy=array([19., 21., 16.,  9.], dtype=float32)>),
              ('BANK_WHEAT',
               <tf.Tensor: shape=(4,), dtype=float32, numpy=array([12., 17., 18., 15.], dtype=float32)>),
              ('BANK_WOOD',
               <tf.Tensor: shape=(4,), dtype=float32, numpy=array([19., 19., 18., 13.], dtype=float32)>),
              ('EDGE(0, 20)_P0_ROAD',
               <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>),
              ('EDGE(0, 20)_P1_ROAD',
           

In [25]:
def preprocess(*fields):
    print(fields)
    features=tf.stack(fields[:-1])
    # convert Target column values to int to make it work for binary classification
    labels=tf.stack([float(x) for x in fields[-1:]])
    return features,labels  # x, y

def preprocess(x, y):
    print(x['BANK_BRICK'])
    print(dir(x['BANK_BRICK']))
    print(type(x))
    print(len(x.keys()))
    print(type(y))
    print(y)
    print(y.keys())
    print(y['VICTORY_POINTS_RETURN'])
    print(tf.transpose(y['VICTORY_POINTS_RETURN']))
    print(tf.reshape(y['VICTORY_POINTS_RETURN'], shape=(4,)))
    labels = tf.stack(y['VICTORY_POINTS_RETURN'])
    return tf.constant([[1], [2], [3]]), tf.reshape(y['VICTORY_POINTS_RETURN'], shape=(BATCH_SIZE,))

train_ds = dataset.map(preprocess).skip(1).take(1).batch(1).repeat()
for element in train_ds:
    print(element)
    break

Tensor("args_0:0", shape=(4,), dtype=float32)
['OVERLOADABLE_OPERATORS', '_USE_EQUALITY', '__abs__', '__add__', '__and__', '__array__', '__array_priority__', '__bool__', '__class__', '__copy__', '__delattr__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__invert__', '__iter__', '__le__', '__len__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmatmul__', '__rmod__', '__rmul__', '__ror__', '__rpow__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__weakref__', '__xor__', '_as_node_def_input', '_as_tf_output', '_c_api_shape', '_consumers', '_create_with_tf_output', '_disallow_bool_casting',

In [23]:
shape = (BATCH_SIZE, 1)
output_units = 1

inputs = tf.keras.Input(shape=shape)
outputs = inputs
outputs = tf.keras.layers.Dense(1, activation=tf.nn.relu)(outputs)
outputs = tf.keras.layers.Dense(units=output_units)(outputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    metrics=["mean_squared_error",],
    optimizer='adam',
    loss="mean_squared_error",
)
model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4, 1)]            0         
_________________________________________________________________
dense_12 (Dense)             (None, 4, 1)              2         
_________________________________________________________________
dense_13 (Dense)             (None, 4, 1)              2         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(train_ds,  steps_per_epoch=1, epochs=1)

1/1 [==============================] - 0s 863us/step - loss: 50.1692 - mean_squared_error: 50.1692
